In [1]:
import pandas as pd

df1 = pd.read_csv('SKW_final_w_Pd.csv')

print("DataFrame Head:")
display(df1.head())

print("\nDataFrame Info:")
display(df1.info())

DataFrame Head:


,Location,UWI,Reading Time,Average Amps (A) (Raw),Drive Frequency (Hz) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,SKW-35,SKW-35,2025-03-01 00:00:00,30.3,40.0,968.9,2045.960600,269.8,342.9,0.3,240.0
1,SKW-35,SKW-35,2025-03-01 00:10:00,29.6,40.0,973.9,2045.950722,270.5,343.0,0.3,240.0
2,SKW-35,SKW-35,2025-03-01 00:20:00,29.9,40.0,974.1,2045.951697,270.1,340.9,0.3,240.0
3,SKW-35,SKW-35,2025-03-01 00:30:00,29.2,40.0,981.0,2045.936217,270.3,339.3,0.3,240.0
4,SKW-35,SKW-35,2025-03-01 00:40:00,29.1,40.0,988.7,2045.932891,271.0,338.9,0.3,240.0



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49685 entries, 0 to 49684
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Location                        49685 non-null  object 
 1   UWI                             49685 non-null  object 
 2   Reading Time                    49685 non-null  object 
 3   Average Amps (A) (Raw)          49685 non-null  float64
 4   Drive Frequency (Hz) (Raw)      49685 non-null  float64
 5   Intake Pressure (psi) (Raw)     49685 non-null  float64
 6   Discharge Pressure (psi) (Raw)  49685 non-null  float64
 7   Intake Temperature (F) (Raw)    49685 non-null  float64
 8   Motor Temperature (F) (Raw)     49685 non-null  float64
 9   Vibration (gravit) (Raw)        49685 non-null  float64
 10  Virtual Rate (BFPD) (Raw)       49685 non-null  float64
dtypes: float64(8), object(3)
memory usage: 4.2+ MB


None

In [2]:
df = df1.drop(df1.columns[4], axis=1)

In [3]:
df['Reading Time'] = pd.to_datetime(df['Reading Time'])
df = df.sort_values(by='Reading Time')
display(df.head())

,Location,UWI,Reading Time,Average Amps (A) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,SKW-35,SKW-35,2025-03-01 00:00:00,30.3,968.9,2045.960600,269.8,342.9,0.3,240.0
1,SKW-35,SKW-35,2025-03-01 00:10:00,29.6,973.9,2045.950722,270.5,343.0,0.3,240.0
2,SKW-35,SKW-35,2025-03-01 00:20:00,29.9,974.1,2045.951697,270.1,340.9,0.3,240.0
3,SKW-35,SKW-35,2025-03-01 00:30:00,29.2,981.0,2045.936217,270.3,339.3,0.3,240.0
4,SKW-35,SKW-35,2025-03-01 00:40:00,29.1,988.7,2045.932891,271.0,338.9,0.3,240.0


In [ ]:
import numpy as np
from scipy.stats import linregress

# --- sort dataframe ---
df = df.sort_values('Reading Time').reset_index(drop=True)

# --- tentukan interval waktu (30 menit) ---
time_interval = pd.Timedelta(minutes=30)

# snap waktu awal & akhir ke kelipatan 30 menit
start_time = df['Reading Time'].iloc[0].floor('30min')
end_time = df['Reading Time'].iloc[-1].ceil('30min')

# buat daftar window tiap 30 menit
time_windows = pd.date_range(start=start_time, end=end_time, freq='30min')

# --- hitung slope ---
slopes_list = []

for window_start in time_windows:
    window_end = window_start + time_interval
    window_df = df[(df['Reading Time'] >= window_start) & (df['Reading Time'] < window_end)]

    # skip kalau data terlalu sedikit
    if len(window_df) < 2:
        continue

    # ambil kolom numerik
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    cols_to_exclude = ['Discharge Pressure (psi) (Raw)']  # contoh kalau mau dikecualikan
    # numerical_cols = [col for col in numerical_cols if col not in cols_to_exclude]

    # hitung slope per kolom
    window_slopes = {}
    for col in numerical_cols:
        temp_df = window_df[['Reading Time', col]].dropna()
        if len(temp_df) > 1:
            temp_time_diff = (temp_df['Reading Time'] - window_start).dt.total_seconds().values
            slope, _, _, _, _ = linregress(temp_time_diff, temp_df[col])
            window_slopes[col + '_slope'] = slope
        else:
            window_slopes[col + '_slope'] = np.nan

    # simpan waktu window
    window_slopes['Window_Start_Time'] = window_start
    slopes_list.append(window_slopes)

# --- konversi ke DataFrame ---
slopes_df = pd.DataFrame(slopes_list)

# jadikan Window_Start_Time sebagai index
slopes_df = slopes_df.set_index('Window_Start_Time')


In [5]:
# from scipy.stats import linregress
# import numpy as np


# # Tentukan interval waktu (30 menit)
# time_interval = pd.Timedelta(hours=0.5)

# # Urutkan DataFrame berdasarkan waktu
# df = df.sort_values('Reading Time').reset_index(drop=True)

# # Buat list semua titik waktu awal yang kelipatan 30 menit dari waktu awal
# start_time = df['Reading Time'].iloc[0]
# end_time = df['Reading Time'].iloc[-1]
# time_windows = pd.date_range(start=start_time, end=end_time, freq='0.5H')

# # Inisialisasi list untuk menyimpan hasil slope
# slopes_list = []

# # Iterasi setiap window waktu
# for window_start in time_windows:
#     window_end = window_start + time_interval
#     window_df = df[(df['Reading Time'] >= window_start) & (df['Reading Time'] < window_end)]

#     # Lewati window jika datanya terlalu sedikit
#     if len(window_df) < 2:
#         continue

#     # Ekstrak kolom numerik
#     numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
#     cols_to_exclude = ['Discharge Pressure (psi) (Raw)']  # jika ingin mengecualikan
#     # numerical_cols = [col for col in numerical_cols if col not in cols_to_exclude]

#     # Hitung slope tiap kolom numerik
#     window_slopes = {}
#     for col in numerical_cols:
#         temp_df = window_df[[col, 'Reading Time']].dropna()
#         if len(temp_df) > 1:
#             temp_time_diff = (temp_df['Reading Time'] - window_start).dt.total_seconds().values
#             slope, _, _, _, _ = linregress(temp_time_diff, temp_df[col])
#             window_slopes[col + '_slope'] = slope
#         else:
#             window_slopes[col + '_slope'] = np.nan

#     window_slopes['Window_Start_Time'] = window_start
#     slopes_list.append(window_slopes)

# # Konversi ke DataFrame
# slopes_df = pd.DataFrame(slopes_list)

# display(slopes_df.head())

In [5]:
# Buat salinan daftar nama kolom
cols1 = slopes_df.columns.tolist()

cols1[0] = 'A'
cols1[1] = 'IP'
cols1[2] = 'DP'   
cols1[3] = 'IT'   
cols1[4] = 'MT'   
cols1[5] = 'V'   
cols1[6] = 'R'

# Terapkan kembali ke dataframe
slopes_df.columns = cols1

In [6]:
df

,Location,UWI,Reading Time,Average Amps (A) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,SKW-35,SKW-35,2025-03-01 00:00:00,30.3,968.9,2045.960600,269.8,342.9,0.3,240.0
1,SKW-35,SKW-35,2025-03-01 00:10:00,29.6,973.9,2045.950722,270.5,343.0,0.3,240.0
2,SKW-35,SKW-35,2025-03-01 00:20:00,29.9,974.1,2045.951697,270.1,340.9,0.3,240.0
3,SKW-35,SKW-35,2025-03-01 00:30:00,29.2,981.0,2045.936217,270.3,339.3,0.3,240.0
4,SKW-35,SKW-35,2025-03-01 00:40:00,29.1,988.7,2045.932891,271.0,338.9,0.3,240.0
...,...,...,...,...,...,...,...,...,...,...
49680,SKW-35,SKW-35,2025-07-10 23:20:00,0.0,1990.3,1546.633414,267.4,268.5,0.2,0.0
49681,SKW-35,SKW-35,2025-07-10 23:30:00,0.0,1990.7,1546.633414,267.4,268.3,0.2,0.0
49682,SKW-35,SKW-35,2025-07-10 23:40:00,0.0,1989.9,1546.633414,267.3,268.3,0.2,0.0
49683,SKW-35,SKW-35,2025-07-10 23:50:00,0.0,1990.1,1546.633414,267.3,268.2,0.2,0.0


In [7]:
# # Pastikan kolom Reading Time sudah datetime
# df1["Reading Time"] = pd.to_datetime(df1["Reading Time"])

# # Tentukan start time dari data pertama
# start_time = df1["Reading Time"].min()

# # Set index menjadi Reading Time
# df_resampled = (
#     df1.set_index("Reading Time")
#        .resample("30T", origin=start_time)
#        .mean(numeric_only=True)
#        .reset_index()
# )
# Pastikan kolom Reading Time sudah datetime
df1["Reading Time"] = pd.to_datetime(df1["Reading Time"])

# Resample ke setiap 30 menit, align ke 00 dan 30 menit
df_resampled = (
    df1.set_index("Reading Time")
       .resample("30T", origin="epoch")   # align ke 00, 30
       .mean(numeric_only=True)
       .reset_index()
)

/var/folders/3s/j5vtskr90z1g_7mq67zd0z6m0000gn/T/ipykernel_10099/228032866.py:20: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample("30T", origin="epoch")   # align ke 00, 30


In [8]:
df_resampled.head()

,Reading Time,Average Amps (A) (Raw),Drive Frequency (Hz) (Raw),Intake Pressure (psi) (Raw),Discharge Pressure (psi) (Raw),Intake Temperature (F) (Raw),Motor Temperature (F) (Raw),Vibration (gravit) (Raw),Virtual Rate (BFPD) (Raw)
0,2025-03-01 00:00:00,29.933333,40.000000,972.300000,2045.954340,270.133333,342.266667,0.300000,240.0
1,2025-03-01 00:30:00,29.200000,40.000000,988.466667,2059.795951,270.966667,339.333333,0.333333,240.0
2,2025-03-01 01:00:00,29.333333,40.000000,1008.666667,2087.523706,273.500000,342.566667,0.400000,240.0
3,2025-03-01 01:30:00,29.300000,40.466667,1025.566667,2087.575195,276.033333,346.833333,0.466667,240.0
4,2025-03-01 02:00:00,29.700000,45.266667,1030.366667,2087.556325,275.833333,347.166667,0.433333,240.0


In [9]:
slopes_df.head()

,A,IP,DP,IT,MT,V,R
Window_Start_Time,,,,,,,
2025-03-01 00:00:00,-0.000333,0.004333,-0.000007,0.000250,-0.001667,0.000000,0.0
2025-03-01 00:30:00,0.000083,0.012250,0.034652,0.001083,0.000417,0.000083,0.0
2025-03-01 01:00:00,0.000000,0.011083,0.000005,0.001917,0.002833,0.000000,0.0
2025-03-01 01:30:00,-0.000250,0.007500,0.000056,0.001000,0.001500,0.000083,0.0
2025-03-01 02:00:00,-0.000083,0.001000,-0.000056,-0.000667,0.000750,-0.000083,0.0


In [10]:
slopes_df = slopes_df.reset_index()
X_predict = slopes_df.drop(columns=['Window_Start_Time'])

# Display the head of the feature matrix
display(X_predict.head())

,A,IP,DP,IT,MT,V,R
0,-0.000333,0.004333,-0.000007,0.000250,-0.001667,0.000000,0.0
1,0.000083,0.012250,0.034652,0.001083,0.000417,0.000083,0.0
2,0.000000,0.011083,0.000005,0.001917,0.002833,0.000000,0.0
3,-0.000250,0.007500,0.000056,0.001000,0.001500,0.000083,0.0
4,-0.000083,0.001000,-0.000056,-0.000667,0.000750,-0.000083,0.0


In [11]:
slopes_df.to_csv('slopes_df_30menit.csv', index=False)
X_predict.to_csv('X_predict_30menit.csv', index=False)
df_resampled.to_csv('df_all.csv', index=False)